In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# 1. Генерація штучних даних
def generate_synthetic_data(num_records=1000):
    np.random.seed(42)
    start_date = datetime(2025, 1, 1)

    cities = ['New York', 'Los Angeles', 'Chicago', 'Dallas', 'Houston', 'Atlanta', 'Miami', 'Phoenix']

    data = {
        'route_id': np.random.randint(101, 120, num_records),
        'start_city': np.random.choice(cities, num_records),
        'end_city': np.random.choice(cities, num_records),
        'date': [start_date + timedelta(days=np.random.randint(0, 365)) for _ in range(num_records)],
        'traffic_index': np.random.uniform(0.5, 5.0, num_records),
        'weather_impact': np.random.choice([0, 0.5, 1.0, 1.5, 2.0], num_records, p=[0.7, 0.1, 0.1, 0.05, 0.05]),
        'fuel_price_usd': np.random.uniform(3.0, 4.5, num_records),
        'social_unrest_flag': np.random.choice([0, 1], num_records, p=[0.95, 0.05]),
    }

    df = pd.DataFrame(data)
    df['delay_hours'] = np.maximum(0,
        np.random.normal(loc=2, scale=1.5, size=num_records) +
        df['traffic_index'] * 0.5 +
        df['weather_impact'] * 2 +
        df['social_unrest_flag'] * 5
    )

    return df

synthetic_data = generate_synthetic_data()

# 2. Моделювання ризиків
X = synthetic_data[['traffic_index', 'weather_impact', 'fuel_price_usd', 'social_unrest_flag']]
y = synthetic_data['delay_hours']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LGBMRegressor(random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)

# 3. Генерація текстового звіту
print("="*60)
print("             ЗВІТ АНАЛІЗУ РИЗИКІВ У ЛАНЦЮГАХ ПОСТАВОК")
print("="*60)
print(f"Дата аналізу: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("-" * 60)

# Додамо передбачені значення затримок до датафрейму
X_test = X_test.copy()
X_test['predicted_delay'] = predictions
X_test['actual_delay'] = y_test

# Додаємо інформацію про маршрут
X_test = X_test.reset_index(drop=True)
route_info = synthetic_data.iloc[X_test.index][['start_city', 'end_city', 'date']].reset_index(drop=True)
analysis_df = pd.concat([route_info, X_test], axis=1)

# Сортування за найбільшою прогнозованою затримкою для виділення найризикованіших маршрутів
top_risky_routes = analysis_df.sort_values(by='predicted_delay', ascending=False).head(5)

print("Топ-5 маршрутів з найбільшим ризиком затримки:")
print("-" * 60)
for index, row in top_risky_routes.iterrows():
    risk_factors = []
    if row['weather_impact'] > 1.0:
        risk_factors.append("несприятливі погодні умови")
    if row['traffic_index'] > 3.0:
        risk_factors.append("висока інтенсивність трафіку")
    if row['social_unrest_flag'] == 1:
        risk_factors.append("соціальні події (протести, страйки)")
    if not risk_factors:
        risk_factors.append("загальні фактори та інциденти")

    print(f"Маршрут: {row['start_city']} -> {row['end_city']}")
    print(f"  Прогнозована затримка: {row['predicted_delay']:.2f} годин")
    print(f"  Ключові фактори ризику: {', '.join(risk_factors)}")
    print("-" * 60)

# Загальна статистика
avg_delay = analysis_df['predicted_delay'].mean()
print(f"Середня прогнозована затримка по всім маршрутам: {avg_delay:.2f} годин")
print("-" * 60)

# Висновки та рекомендації
print("Висновки та рекомендації:")
print("Модель виділила ключові фактори ризику, що суттєво впливають на затримки. Ухвалення рішень в реальному часі, таких як зміна маршруту в обхід зон з високим трафіком або несприятливими погодними умовами, допоможе мінімізувати потенційні втрати.")
print("-" * 60)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000289 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 517
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 4
[LightGBM] [Info] Start training from score 4.306833
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in